# Trees

In [88]:
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import netCDF4 as n
%matplotlib inline

## Read the data

Working with netcdf files.

http://unidata.github.io/netcdf4-python/

### Norway

We are not going to use these data (we use files for Europe).

In [89]:
path = "./files/"
filenames = ["specific_humidity_Norway.nc", "relative_humidity_Norway.nc", "pressure_Norway.nc",  
             "temperature_Norway.nc", "total_cloud_cover_Norway.nc"]


cloud = n.Dataset(path + filenames[-1], "r")
relative = n.Dataset(path + filenames[1], "r")
specific = n.Dataset(path + filenames[0], "r")
pressure = n.Dataset(path + filenames[2], "r")
temperature = n.Dataset(path + filenames[3], "r")

files_nor = {'cloud':cloud, 'relative':relative, 'specific':specific, 'pressure':pressure, 'temperature':temperature}

In [90]:
#print(pressure.variables)
tcc_nor = cloud.variables["tcc"]
rh_nor = relative.variables["r"]
# level = relative.variables["level"]
q_nor = specific.variables["q"]
sp_nor = pressure.variables["sp"]
t2m_nor = temperature.variables["t2m"]

### Europe
We use only the first time stamp, and level 1000 hPa.

#### Open files

In [91]:
path = "./files/"
filenames = ["specific_humidity_Europa_sp.nc", "relative_humidity_Europa_sp.nc", "pressure_Europa_sp.nc",  
             "temperature_Europa_sp.nc", "total_cloud_cover_Europa_sp.nc"]


cloud = n.Dataset(path + filenames[-1], "r")
relative = n.Dataset(path + filenames[1], "r")
specific = n.Dataset(path + filenames[0], "r")
pressure = n.Dataset(path + filenames[2], "r")
temperature = n.Dataset(path + filenames[3], "r")

my_files = {'cloud':cloud, 'relative':relative, 'specific':specific, 'pressure':pressure, 'temperature':temperature}

#### Variables

In [92]:
for key, value in my_files.items():
    print(key, ': ', value.variables)

cloud :  OrderedDict([('longitude', <class 'netCDF4._netCDF4.Variable'>
float32 longitude(longitude)
    units: degrees_east
    long_name: longitude
unlimited dimensions: 
current shape = (77,)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('latitude', <class 'netCDF4._netCDF4.Variable'>
float32 latitude(latitude)
    units: degrees_north
    long_name: latitude
unlimited dimensions: 
current shape = (61,)
filling on, default _FillValue of 9.969209968386869e+36 used
), ('time', <class 'netCDF4._netCDF4.Variable'>
int32 time(time)
    units: hours since 1900-01-01 00:00:0.0
    long_name: time
    calendar: gregorian
unlimited dimensions: time
current shape = (32,)
filling on, default _FillValue of -2147483647 used
), ('tcc', <class 'netCDF4._netCDF4.Variable'>
int16 tcc(time, latitude, longitude)
    scale_factor: 1.525948758640685e-05
    add_offset: 0.49999237025720666
    _FillValue: -32767
    missing_value: -32767
    units: (0 - 1)
    long_name: Total cloud co

In [93]:
tcc = cloud.variables["tcc"]
rh = relative.variables["r"]
q = specific.variables["q"]
sp = pressure.variables["sp"]
t2m = temperature.variables["t2m"]

my_vars = {'tcc':tcc, 'rh':rh, 'q':q, 'sp':sp, 't2m':t2m}

#### Dimensions and sizes?

In [94]:
for key, value in my_files.items():
    print(key, ': ', value.dimensions)

cloud :  OrderedDict([('longitude', <class 'netCDF4._netCDF4.Dimension'>: name = 'longitude', size = 77
), ('latitude', <class 'netCDF4._netCDF4.Dimension'>: name = 'latitude', size = 61
), ('time', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 32
)])
relative :  OrderedDict([('longitude', <class 'netCDF4._netCDF4.Dimension'>: name = 'longitude', size = 77
), ('latitude', <class 'netCDF4._netCDF4.Dimension'>: name = 'latitude', size = 61
), ('level', <class 'netCDF4._netCDF4.Dimension'>: name = 'level', size = 6
), ('time', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 32
)])
specific :  OrderedDict([('longitude', <class 'netCDF4._netCDF4.Dimension'>: name = 'longitude', size = 77
), ('latitude', <class 'netCDF4._netCDF4.Dimension'>: name = 'latitude', size = 61
), ('level', <class 'netCDF4._netCDF4.Dimension'>: name = 'level', size = 6
), ('time', <class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 32
)])
pressure

In [95]:
for key, value in my_vars.items():
    print(key, ': ', value.dimensions)

tcc :  ('time', 'latitude', 'longitude')
rh :  ('time', 'level', 'latitude', 'longitude')
q :  ('time', 'level', 'latitude', 'longitude')
sp :  ('time', 'latitude', 'longitude')
t2m :  ('time', 'latitude', 'longitude')


#### Levels

In [96]:
print('rh levels: ', relative['level'][:])
print('q levels: ', specific['level'][:])

rh levels:  [1000  850  700  500  400  300]
q levels:  [1000  850  700  500  400  300]


In [97]:
print('Units for the latitude:', relative.variables['latitude'].units)
print('Units for the longitude:', relative.variables['longitude'].units)
print('Units for the time:', relative.variables['time'].units)
print('Units for the level:', relative.variables['level'].units)

Units for the latitude: degrees_north
Units for the longitude: degrees_east
Units for the time: hours since 1900-01-01 00:00:0.0
Units for the level: millibars


## Correct all the variables for offset and scale_factor

In [98]:
for key, value in my_vars.items():
    #my_vars[key] = value.scale
    my_vars[key] = value.add_offset
    print(my_vars[key])

0.49999237025720666
56.78843174459784
0.006964579015321184
91282.2142899379
272.00388021345276


##### Extraxt data for 1000 hPa

In [59]:
my_vars['tcc'] = tcc.scale.add_offset[:, :, :]
my_vars['rh'] = rh.add_offset[:, 1, :, :]
my_vars['q'] = q.add_offset[:, 1, :, :]
my_vars['sp'] = sp.add_offset[:, :, :]
my_vars['t2m'] = rh.add_offset[:, :, :]



AttributeError: 'bool' object has no attribute 'add_offset'